In [50]:
#importing libraries
import pandas as pd
import plotly.graph_objects as go
import numpy as np

#sklearn imports
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [51]:
#Initiliazing the data and transferring to pandas
#for easier access and manipulation

#the dataset is given as an array
#but the call itself is of the form
#sklearn.utils._bunch.Bunch
#which is functionaly similar to a dictionary
cali_array= sklearn.datasets.fetch_california_housing()

#the target value 'MedHouseVal' is under 'target' and not the data
#print(cali_array)

cali_df= pd.DataFrame(cali_array.data,
                       columns= cali_array.feature_names)

cali_df['MedHouseVal']= cali_array.target

cali_df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


1. Load and Explore the Data

In [52]:
#Plot a scatter plot of MedInc vs. MedHouseVal to visualize their relationship.

fig= go.Figure()

#MedHouseVal is the target so it is the dependent variable
fig.add_trace(go.Scatter(
    x= cali_df['MedInc'],
    y= cali_df['MedHouseVal'],
    mode= 'markers'
))

#need to change the size of the plot
#20000 points in the original scale was terrible
fig.update_layout(
    title= 'Median Income vs. Median House Value',
    xaxis_title= 'Median Income (MedInc)',
    yaxis_title='Median House Value (MedHouseVal)',
    width= 1000,  
    height= 700   
)

fig.show()

In [53]:
#Calculate summary statistics (mean, median, standard deviation) for both variables.

#dictionary form makes it easier to access
stats_medinc= {'mean': np.mean(cali_df['MedInc']),
               'median': np.median(cali_df['MedInc']),
               'sd': np.std(cali_df['MedInc'])}

stats_medval= {'mean': np.mean(cali_df['MedHouseVal']),
               'median': np.median(cali_df['MedHouseVal']),
               'sd': np.std(cali_df['MedHouseVal'])}

#could put both of them in a same dataframe for easier visualization
#task is simple enough to just use these though
print(f"medinc: {stats_medinc}")
print(f"medval: {stats_medval}")


medinc: {'mean': 3.8706710029069766, 'median': 3.5347999999999997, 'sd': 1.899775694574878}
medval: {'mean': 2.068558169089147, 'median': 1.797, 'sd': 1.1539282040412349}


2. Preprocess the Data

In [54]:
#Split the data into training (80%) and testing (20%) sets using train_test_split

#standardization is used to transform the data into a normal distribution
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler = StandardScaler()

#we scale only the training data, not the target
#the scale is then transfered over to the testing set
#using fittransform
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

#to run the fitransform for a single feature
#need to "Reshape your data either using array.reshape(-1, 1)" - error message
x= cali_df['MedInc']
x= x.to_numpy().reshape(-1, 1)
x_scaled = scaler.fit_transform(x)

y= cali_df['MedHouseVal']

#fixing random state for reproducability
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y,
                                                    test_size= 0.2,
                                                    random_state= 5)

#note, the mean and std of x_caled are not perfectly 0 and 1 due
#to a problem with numpy, but they are extremely close to those values

3. Build a Linear Regression Model

In [55]:
#Train a linear regression model on the training data using
#both (Batch)1 Gradient Descent and Stochastic Gradient Descent

#Stochastic Gradient Descent model
#mse is used as it is also used on the batch gradient desecent
sgd_model = SGDRegressor(early_stopping= False,
                         random_state= 5,
                         loss="squared_error",
                         max_iter= 500)

sgd_model.fit(x_train, y_train)

SGDRegressor(max_iter=500, random_state=5)

In [56]:
#Batch Gradient Descent

#as far as I understood, this one doesn't have a
#import from sklearn. This needs to eb done manually.
#this code was done using as reference:
#https://spotintelligence.com/2024/05/15/batch-gradient-descent/
#https://medium.com/@jaleeladejumo/gradient-descent-from-scratch-batch-gradient-descent-stochastic-gradient-descent-and-mini-batch-def681187473
#which are all credited in the report

#the batch gradient descent will be done using mse
#as it is easier, there are more resources online concerning
#understanding it and we have used mse during the lectures

def batch_gd(x, y, step_size= 0.01):

    #need to add a column for the itnercept term, as the
    #original train split only has the original features
    x_concat= np.c_[np.ones((len(x), 1)), x]

    #we need n as it is the number of features
    #to create the array containing the coefficients
    m, n= x_concat.shape

    #theta are the coeffiecients and intercept of the prediction
    theta= np.zeros([n, 1])
    losses= []

    y= y.to_numpy().reshape(-1, 1)

    #similarly to gcd, we define a max number of iterations
    #set to 100 to match the gcd from previous block of code
    for iteration in range(1000):
        y_pred= x_concat.dot(theta)

        #how much the prediction deviates from the intended
        loss= np.mean((y - y_pred)**2)

        gradient= 2/m * x_concat.T.dot(x_concat.dot(theta) - y)

        #multiply each theta by the value of the learning rate
        #is the rate at which it converges
        #ideally needs to be optimized as a hyperparameter
        #but will use a fixed value for this exercise
        theta -= step_size * gradient
        
        losses.append(loss)

    return theta, losses

In [57]:
#plotting to make sure it is working as intented
#running the bgd

theta_bgd, loss_bgd = batch_gd(x= x_train,
                    y= y_train)

fig= go.Figure()

fig.add_trace(go.Scatter(
    x= list(range(500)),
    y= loss_bgd,
    mode= 'lines'))

fig.update_layout(
    title='Cost Function (MSE) with each iteration',
    xaxis_title='Iterations',
    yaxis_title='Cost (MSE)',
    width= 1000,
    height= 750)

fig.show()


4. Make Predictions and 5. Visualize the Results

In [58]:
#Predict house values for the test set.

#for the bgd, the y_pred is given by the dot product
#need to add an intercep to x_test as well
x_test_concat= np.c_[np.ones((x_test.shape[0], 1)), x_test]

#bgd
y_pred_bgd= x_test_concat.dot(theta_bgd)
#sgd
y_pred_sgd= sgd_model.predict(x_test)

y_pred_bgd

array([[1.59175535],
       [1.95656083],
       [1.33682953],
       ...,
       [1.65165937],
       [1.70735009],
       [1.30099558]])

In [59]:
#Plot the regression line overlaid on the test data scatter plot.

fig = go.Figure()

#need to make the bgd a 1d array
#for it to be plotted
y_pred_bgd= y_pred_bgd.flatten()

#original data points
fig.add_trace(go.Scatter(
    x=x_test[:, 0],
    y=y_test,
    mode='markers',
    name='Original data points',
    marker=dict(color='blue', opacity=0.6)
))

#bgd line
#locates the index
#need to use the same index for x_test and y_pred
#or else it doesn't make sense
sorted_indices = np.argsort(x_test[:, 0])
#print(y_pred_bgd[sorted_indices])
fig.add_trace(go.Scatter(
    x=x_test[sorted_indices, 0],
    y=y_pred_bgd[sorted_indices],
    mode='lines',
    name='Batch Gradient Descent',
))

#sgd line
fig.add_trace(go.Scatter(
    x=x_test[sorted_indices, 0],
    y=y_pred_sgd[sorted_indices],
    mode='lines',
    name='Stochastic Gradient Descent',
))

fig.update_layout(
    title="Comparison of SGD vs BGD model using only Median Income",
    xaxis_title="Median Income (MedInc)",
    yaxis_title="Median House Value",
    legend=dict(x=0.05, y=0.95),
    width= 1000,
    height= 750
)

fig.show()



In [ ]:
#analizing the accuracy of the models

results = {}

results["Batch Gradient Descent"] = {
    "MSE": mean_squared_error(y_test, y_pred_bgd),
    "R squared": r2_score(y_test, y_pred_bgd) 
    }

results["Stochastic Gradient Descent"] = {
    "MSE": mean_squared_error(y_test, y_pred_sgd),
    "R squared": r2_score(y_test, y_pred_sgd) 
    }

results

#yeah the accuracy between the 2 is extremely similar
#even though it was already visible from the 2 lines prior

{'Batch Gradient Descent': {'MSE': 0.7106636942755551,
  'R squared': 0.48490837612706916},
 'Stochastic Gradient Descent': {'MSE': 0.7121790926222216,
  'R squared': 0.48381000990759526}}

6. Rerun the experiment but for more features

In [62]:
x_all= cali_df.drop(columns= ["MedHouseVal"])
#no need for reshape since dim >1
x_scaled_all = scaler.fit_transform(x_all)

y_all= cali_df['MedHouseVal']

#fixing random state for reproducability
x_train_all, x_test_all, y_train_all, y_test_all = train_test_split(x_scaled_all, y_all,
                                                    test_size= 0.2,
                                                    random_state= 5)

#note, the mean and std of x_caled are not perfectly 0 and 1 due
#to a problem with numpy, but they are extremely close to those values

In [63]:
#sgd
sgd_model_all = SGDRegressor(early_stopping= False,
                         random_state= 5,
                         loss="squared_error",
                         max_iter= 1000)

sgd_model_all.fit(x_train_all, y_train_all)

#bgd
theta_bgd_all, loss_bgd_all = batch_gd(x= x_train_all,
                    y= y_train_all)

In [64]:
x_test_concat_all= np.c_[np.ones((x_test_all.shape[0], 1)), x_test_all]

#bgd
y_pred_bgd_all= x_test_concat_all.dot(theta_bgd_all)
#sgd
y_pred_sgd_all= sgd_model_all.predict(x_test_all)

np.min(y_pred_sgd_all)

-1425034.6593335832

In [65]:
fig = go.Figure()

#need to make the bgd a 1d array
#for it to be plotted
y_pred_bgd_all= y_pred_bgd_all.flatten()

#original data points
fig.add_trace(go.Scatter(
    x=x_test_all[:, 0],
    y=y_test_all,
    mode='markers',
    name='Original data points',
))

#bgd line
#locates the index
#need to use the same index for x_test and y_pred
#or else it doesn't make sense
sorted_indices_all = np.argsort(x_test_all[:, 0])
#print(y_pred_bgd[sorted_indices])
fig.add_trace(go.Scatter(
    x=x_test_all[sorted_indices_all, 0],
    y=y_pred_bgd_all[sorted_indices_all],
    mode='lines',
    name='Batch Gradient Descent',
))

#sgd line
fig.add_trace(go.Scatter(
    x=x_test_all[sorted_indices_all, 0],
    y=y_pred_sgd_all[sorted_indices_all],
    mode='lines',
    name='Stochastic Gradient Descent',
))

fig.update_layout(
    title="Comparison of SGD vs BGD model using only Median Income",
    xaxis_title="Median Income (MedInc)",
    yaxis_title="Median House Value",
    width= 1000,
    height= 750
)

fig.show()

#Clearly, the SGD does not fit well with the data
#I was unable to identify what is the reason behind this
#but it could be due to the amount of outliers
#in the dataset